## class

In [7]:
from utils.data_prepare import load_class_list
class_list = load_class_list()
len(class_list)

102

## data

In [1]:
from utils.datasets import get_dataloader
train_loader, val_loader = get_dataloader()

In [2]:
for i_batch, sample_batched in enumerate(train_loader):
    print("batch: ", i_batch)
    print("img.shape: ", sample_batched[0].shape)
    print("label.shape: ", sample_batched[1].shape)
    break

batch:  0
img.shape:  torch.Size([32, 3, 224, 224])
label.shape:  torch.Size([32])


## model

In [3]:
import torchvision.models as models

resnet50 = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)

In [5]:
resnet50.fc

Linear(in_features=2048, out_features=1000, bias=True)

In [8]:
import torch.nn as nn
resnet50.fc = nn.Linear(2048,102)
nn.init.xavier_normal_(resnet50.fc.weight)

Parameter containing:
tensor([[ 0.0570,  0.0672,  0.0206,  ..., -0.0104, -0.0206, -0.0793],
        [ 0.0071,  0.0121, -0.0378,  ...,  0.0190,  0.0385, -0.0190],
        [-0.0229, -0.0401,  0.0095,  ..., -0.0403,  0.0005, -0.0602],
        ...,
        [-0.0473,  0.0435,  0.0457,  ..., -0.0287,  0.0418,  0.0169],
        [ 0.0253,  0.0187, -0.0010,  ..., -0.0160,  0.0155, -0.0023],
        [-0.0503, -0.0725,  0.0680,  ...,  0.0709,  0.0063,  0.0205]],
       requires_grad=True)

In [9]:
for param in resnet50.parameters():
    param.requires_grad = False

for param in resnet50.fc.parameters():
    param.requires_grad = True

In [10]:
from torchinfo import summary
summary(resnet50, input_size=(1, 3, 224, 224))

Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   [1, 102]                  --
├─Conv2d: 1-1                            [1, 64, 112, 112]         (9,408)
├─BatchNorm2d: 1-2                       [1, 64, 112, 112]         (128)
├─ReLU: 1-3                              [1, 64, 112, 112]         --
├─MaxPool2d: 1-4                         [1, 64, 56, 56]           --
├─Sequential: 1-5                        [1, 256, 56, 56]          --
│    └─Bottleneck: 2-1                   [1, 256, 56, 56]          --
│    │    └─Conv2d: 3-1                  [1, 64, 56, 56]           (4,096)
│    │    └─BatchNorm2d: 3-2             [1, 64, 56, 56]           (128)
│    │    └─ReLU: 3-3                    [1, 64, 56, 56]           --
│    │    └─Conv2d: 3-4                  [1, 64, 56, 56]           (36,864)
│    │    └─BatchNorm2d: 3-5             [1, 64, 56, 56]           (128)
│    │    └─ReLU: 3-6                    [1, 64, 56, 56]    

## train

In [12]:
import torch

def score(model, dataloader, device):
    y_true = []
    y_pred = []
    with torch.no_grad():
        for x, y in dataloader:
            x = x.to(device)
            y = y.to(device)
            out = model(x)
            y_pred.extend(torch.argmax(out, dim=-1).tolist())
            y_true.extend(y.tolist())
    accuracy = sum([1 if y_true[i] == y_pred[i] else 0 for i in range(len(y_true))]) / len(y_true)
    return accuracy

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet50.to(device)

import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet50.fc.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)

In [14]:
num_epochs = 10
for epoch in range(num_epochs):
    # Train
    for i, (data, labels) in enumerate(train_loader):
        data = data.to(device)
        labels = labels.to(device)

        outputs = resnet50(data)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()
        if i % 20 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                .format(epoch+1, num_epochs, i+1, len(train_loader), loss.item()))
            
    # Evaluate
    train_acc = score(resnet50, train_loader, device)
    val_acc = score(resnet50, val_loader, device)
    print("Epoch [{}/{}], Train Acc: {:.4f}, Val Acc: {:.4f}".format(epoch+1, num_epochs, train_acc, val_acc))

    # Save the model checkpoint
    if (epoch+1) % 2 == 0:
        torch.save(resnet50.state_dict(), 'resnet50_epoch{}.pth'.format(epoch+1))

    # Decay learning rate
    scheduler.step()

Epoch [1/10], Step [1/225], Loss: 4.7531
Epoch [1/10], Step [21/225], Loss: 3.8426


KeyboardInterrupt: 